In [1]:
# These two lines are necessary only if GemPy is not installed
import sys, os
sys.path.append("../../..")

# Importing GemPy
import gempy as gp

# Embedding matplotlib figures in the notebooks
%matplotlib inline

# Importing auxiliary libraries
import numpy as np
import matplotlib.pyplot as plt

import theano
import theano.tensor as T
import pymc3 as pm

In [2]:
theano.__path__

['/anaconda3/envs/gempy/lib/python3.7/site-packages/theano']

#### Testing reduce dimension

This is the behaviour we need to replicate on theano:

In [3]:
a = np.arange(24).reshape(-1,2)
b = [4,2]

np.add.reduceat(a, np.r_[0, np.cumsum(b)[:-1]]) 


array([[ 12,  16],
       [120, 128]])

#### Testing gradient of repeat

In [4]:
T.alloc([5, 2, 4], 5).eval()

array([5, 2, 4])

In [5]:
a = theano.shared(np.arange(6).reshape(-1,2))
l = theano.tensor.as_tensor([0,1])

In [6]:
T.as_tensor(3)

TensorConstant{3}

In [7]:

def repeat_list(val, r_0, r_1, ref_points):
#     val= theano.printing.Print()(val)
#     rep = theano.printing.Print()(rep)
    ref_points = T.set_subtensor(ref_points[r_0: r_1], T.alloc(val, r_1))
    return ref_points

ref_points_init = T.zeros((6, 2))
ref_points = theano.scan(repeat_list,
                         outputs_info=[ref_points_init],
                         sequences=[a, dict(input=l, taps=[0, 1])],
                         return_list=False)

In [8]:
ref_points[0].eval()

array([[[0., 1.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]]])

In [9]:
x = T.scalar()


In [10]:
a = theano.shared(np.arange(6).reshape(-1,2), )

In [11]:
a.eval()

array([[0, 1],
       [2, 3],
       [4, 5]])

In [12]:
b = T.repeat(a**x, [2,5,6], axis=0)
b.eval({x:5})

array([[0.000e+00, 1.000e+00],
       [0.000e+00, 1.000e+00],
       [3.200e+01, 2.430e+02],
       [3.200e+01, 2.430e+02],
       [3.200e+01, 2.430e+02],
       [3.200e+01, 2.430e+02],
       [3.200e+01, 2.430e+02],
       [1.024e+03, 3.125e+03],
       [1.024e+03, 3.125e+03],
       [1.024e+03, 3.125e+03],
       [1.024e+03, 3.125e+03],
       [1.024e+03, 3.125e+03],
       [1.024e+03, 3.125e+03]])

In [13]:
T.grad(b[-1, -1], x).eval({x:5})

ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: Join(TensorConstant{0}, TensorConstant{(1, 1) of 0}, TensorConstant{[[2 5 6]]})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/anaconda3/envs/gempy/lib/python3.7/site-packages/theano/gof/opt.py", line 2034, in process_node
    replacements = lopt.transform(node)
  File "/anaconda3/envs/gempy/lib/python3.7/site-packages/theano/tensor/opt.py", line 6518, in constant_folding
    required = thunk()
  File "/anaconda3/envs/gempy/lib/python3.7/site-packages/theano/gof/op.py", line 862, in rval
    thunk()
  File "/anaconda3/envs/gempy/lib/python3.7/site-packages/theano/gof/cc.py", line 1739, in __call__
    reraise(exc_type, exc_value, exc_trace)
  File "/anaconda3/envs/gempy/lib/python3.7/site-packages/six.py", line 693, in reraise
    raise value
ValueError: all the input array dimensions except for the concatenation axis must match exa

1


ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: Join(TensorConstant{0}, TensorConstant{(1, 1) of 0}, TensorConstant{[[2 5 6]]})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/anaconda3/envs/gempy/lib/python3.7/site-packages/theano/gof/opt.py", line 2034, in process_node
    replacements = lopt.transform(node)
  File "/anaconda3/envs/gempy/lib/python3.7/site-packages/theano/tensor/opt.py", line 6518, in constant_folding
    required = thunk()
  File "/anaconda3/envs/gempy/lib/python3.7/site-packages/theano/gof/op.py", line 862, in rval
    thunk()
  File "/anaconda3/envs/gempy/lib/python3.7/site-packages/theano/gof/cc.py", line 1739, in __call__
    reraise(exc_type, exc_value, exc_trace)
  File "/anaconda3/envs/gempy/lib/python3.7/site-packages/six.py", line 693, in reraise
    raise value
ValueError: all the input array dimensions except for the concatenation axis must match exa

ValueError: all the input array dimensions except for the concatenation axis must match exactly
Apply node that caused the error: Join(TensorConstant{0}, TensorConstant{(1, 1) of 0}, TensorConstant{[[2 5 6]]})
Toposort index: 7
Inputs types: [TensorType(int8, scalar), TensorType(int8, row), TensorType(int64, row)]
Inputs shapes: [(), (1, 1), (1, 3)]
Inputs strides: [(), (1, 1), (24, 8)]
Inputs values: [array(0, dtype=int8), array([[0]], dtype=int8), array([[2, 5, 6]])]
Outputs clients: [[CumOp{None, add}(Join.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/anaconda3/envs/gempy/lib/python3.7/site-packages/theano/gradient.py", line 1326, in access_grad_cache
    term = access_term_cache(node)[idx]
  File "/anaconda3/envs/gempy/lib/python3.7/site-packages/theano/gradient.py", line 1021, in access_term_cache
    output_grads = [access_grad_cache(var) for var in node.outputs]
  File "/anaconda3/envs/gempy/lib/python3.7/site-packages/th

In [ ]:
T.as_tensor_variable([2, 4, 5])[0].eval()

In [ ]:
c.eval({x:5})

In [ ]:
c = T.repeat(a**x, 5, axis=0)
T.grad(c[5, 1], x).eval({x:5})

In [ ]:
T.grad(b[0,0], a).eval({x:5})

In [ ]:
%debug

In [ ]:
path_to_data = os.pardir+"/../data/input_data/jan_models/"

geo_data = gp.create_data([0,1000,0,1000,0,1000],resolution=[20,2,200], 
                        path_o = path_to_data + "model1_orientations.csv",
                        path_i = path_to_data + "model1_surface_points.csv") 

In [ ]:
gp.plot.plot_data(geo_data)

In [ ]:
from gempy.core.interpolator import InterpolatorWeights

In [ ]:
iw = InterpolatorWeights(geo_data.surface_points, geo_data.orientations, geo_data.grid, geo_data.surfaces,
                    geo_data.series, geo_data.faults, geo_data.additional_data)

In [ ]:
iw.theano_graph.input_parameters_kriging

In [ ]:
geo_model_T = theano.OpFromGraph(iw.theano_graph.input_parameters_kriging,
                                 [iw.theano_graph.compute_weights()], inline=False,
                                 on_unused_input='warn',
                               name='geo_model')

In [ ]:
iw.set_theano_shared_kriging()
iw.set_theano_shared_nuggets()
iw.set_theano_shared_structure_surfaces()

In [ ]:
iw.compile_th_fn()

In [ ]:
geo_model_T(*iw.get_python_input_weights())

In [ ]:
iw.theano_graph.input_parameters_kriging

In [ ]:
iw.get_python_input_weights()

In [ ]:
iw.theano_graph.len_points

In [ ]:
iw.theano_graph.number_of_points_per_surface_T.get_value()

In [ ]:
th_f = theano.function([], geo_model_T(*iw.get_python_input_weights()))

In [ ]:
th_f()

In [ ]:
input_sh = []
for i in iw.get_python_input_weights():
    input_sh.append(theano.shared(i))

In [ ]:
b = geo_data.interpolator.theano_graph.input_parameters_kriging

In [ ]:
a = geo_model_T(*b).sum()

In [ ]:
theano.grad(a, b)

## Compiling gempy with PyMC3

In [ ]:
import theano
import theano.tensor as T
#theano.config.compute_test_value = 'ignore'

In [ ]:
i = geo_data.interpolator.get_python_input_block()

In [ ]:
geo_model_T = theano.OpFromGraph(geo_data.interpolator.theano_graph.input_parameters_loop,
                               geo_data.interpolator.theano_graph.compute_series(), inline=False,
                                 on_unused_input='warn',
                               name='geo_model')

In [ ]:
T.grad(geo_model_T(*geo_data.interpolator.theano_graph.input_parameters_loop)[2][0], 
       geo_data.interpolator.theano_graph.input_parameters_loop[3])

In [ ]:
th_f = theano.function([], geo_model_T(*i))

In [ ]:
th_f()

In [ ]:
theano.config.compute_test_value = 'ignore'

geo_model_T(*geo_data.interpolator.theano_graph.input_parameters_loop)

In [ ]:
 geo_data.interpolator.theano_graph.input_parameters_loop[0]

In [ ]:
th_f = theano.function([], geo_model_T(*i))

In [ ]:
th_f()

In [ ]:
rf = geo_data.rescaling.df.loc['values', 'rescaling factor']
centers = geo_data.rescaling.df.loc['values', 'centers']

In [ ]:
import theano
import theano.tensor as tt
theano.config.compute_test_value = 'warn'
# We convert a python variable to theano.shared
input_sh = []
for ii in i:
    input_sh.append(theano.shared(ii))

with pm.Model() as model:
    r2 = pm.Normal('rock2', 600, 50)
    r1 = pm.Normal('rock1', 400, 50)
    val2 = (r2 - centers[2]) / rf + 0.5001
    val1 = (r1 - centers[2]) / rf + 0.5001

    input_sh[4] = tt.set_subtensor(input_sh[4][g.groups[1], 2], val2)
    input_sh[4] = tt.set_subtensor(input_sh[4][g.groups[2], 2], val1)
    
    # we have to take the sol 0
    geo = geo_model_T(*input_sh)[0][0][0:8000]
    well = geo.reshape(geo_data.grid.regular_grid.resolution)[10,1,:]
    thickness = pm.Deterministic('thickness', tt.sum(tt.eq(well, 2)) * geo_data.grid.regular_grid.dz)
    thickness.name = 'thickness'
    a = pm.Normal('y', mu=thickness, sd=20, observed=120)
    b = pm.NUTS()
    trace = pm.sample(10000, chains=1,
                     step =b,
                     compute_convergence_checks=True, live_plot=False)

In [ ]:
pm.traceplot(trace)